### save the metrics (static ones; not lss ones) for v1.4 dbs.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import os
import pandas as pd
%matplotlib inline

In [2]:
data_dir = '/global/cscratch1/sd/awan/lsst_output/post_wp_output_v1.4/summary_data/'
outdir = '/global/homes/a/awan/LSST/lsstRepos/ObsStrat/postwp/paper-data/summary_csv_v1.4/'
os.makedirs(outdir, exist_ok=True)

### read in the data for the static metrics and store as a dictionary

In [3]:
# read in the data 
data = {}
yr_label = {}
for yr in [1, 3, 6, 10]:
    files = [f for f in os.listdir( data_dir ) if f.endswith('csv') and f.__contains__('y%s_' % yr) and f.startswith('eg_') ]
    for file in files:
        print( file )
        key = 'yr%s_%s' % (yr, file.split('_')[4])
        data[ key ] = pd.read_csv('%s/%s' % (data_dir, file))
        yr_label[ key ] =  r'Y%s (i$>$%s) ' % (yr, file.split('_')[4].split('limi')[-1])
        
print( yr_label )
print(data.keys())
print(data[list(data.keys())[0]].keys() )

eg_footprint_stats_y1_limi24.75_nodither_nside256.csv
eg_footprint_stats_y3_limi25.35_nodither_nside256.csv
eg_footprint_stats_y6_limi25.72_nodither_nside256.csv
eg_footprint_stats_y10_limi26.0_nodither_nside256.csv
{'yr1_limi24.75': 'Y1 (i$>$24.75) ', 'yr3_limi25.35': 'Y3 (i$>$25.35) ', 'yr6_limi25.72': 'Y6 (i$>$25.72) ', 'yr10_limi26.0': 'Y10 (i$>$26.0) '}
dict_keys(['yr1_limi24.75', 'yr3_limi25.35', 'yr6_limi25.72', 'yr10_limi26.0'])
Index(['dbname', 'Area (deg2)', '$i$-band depth: median',
       ' $i$-band depth: std'],
      dtype='object')


In [4]:
# read in the static metrics
for key in data:
    yr_tag = key.split('_')[0].split('yr')[-1]
    lim_tag = key.split('_')[1].split('limi')[-1]
    print(yr_tag, lim_tag)

    data[key] = data[key].rename(columns={'Area (deg2)': 'Y%s effective survey area ' % (yr_tag),
                                          '$i$-band depth: median': 'median Y%s $i$-band coadded depth in effective survey area' % (yr_tag),
                                          ' $i$-band depth: std': 'std in Y%s $i$-band coadded depth in effective survey area' % (yr_tag)} )
    #fname = 'lss_metrics_%s.csv' % key
    #data[key].to_csv('%s/%s' % (outdir, fname), index=False)

1 24.75
3 25.35
6 25.72
10 26.0


In [5]:
# just check things
data['yr1_limi24.75']

,dbname,Y1 effective survey area,median Y1 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area
0,short_exp_2ns_1expt_v1.4_10yrs,14683.44,25.10,0.14
1,footprint_bluer_footprintv1.4_10yrs,14344.58,25.06,0.15
2,pair_strategy_0_v1.4_10yrs,14737.89,25.08,0.14
3,wfd_depth_scale0.95_noddf_v1.4_10yrs,14989.73,25.17,0.16
4,bulges_i_heavy_v1.4_10yrs,15447.46,25.01,0.14
...,...,...,...,...
70,wfd_depth_scale0.85_noddf_v1.4_10yrs,15029.97,25.14,0.15
71,pair_strategy_3_v1.4_10yrs,14615.35,25.07,0.14
72,footprint_add_mag_cloudsv1.4_10yrs,14841.23,25.08,0.14
73,wfd_depth_scale0.85_v1.4_10yrs,14582.20,25.07,0.15


#### read in the `list.csv` file: gives us the order of the 

In [6]:
given_list = pd.read_csv('%s/order_list.csv' % outdir, )
given_list

,name
0,agnddf_v1.4_10yrs.db
1,alt_roll_mod2_dust_sdf_0.20_v1.4_10yrs.db
2,baseline_2snapsv1.4_10yrs.db
3,baseline_v1.4_10yrs.db
4,bulges_bs_v1.4_10yrs.db
...,...
70,wfd_depth_scale0.90_v1.4_10yrs.db
71,wfd_depth_scale0.95_noddf_v1.4_10yrs.db
72,wfd_depth_scale0.95_v1.4_10yrs.db
73,wfd_depth_scale0.99_noddf_v1.4_10yrs.db


In [7]:
# check some more things:
len( list( given_list['name'] ) ), len( np.unique( given_list['name'] ) )

(75, 75)

#### find the indices to get the order

In [8]:
# find the indices to get the order
ind = []
for db in given_list['name']:
    ind_temp = np.where( data['yr1_limi24.75']['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

[40  6 15 63 25 53 30 43 20  4 51 36 45 27 66 58 18 72 19 33 12  1 54 60
 32  7 26 67  2 28 65 71 44 38 39 37 13 46 57 34  0  9 23 24 56 16 49  5
 69 74 42 29 14  8 52 64 48 68 31 62 35 21 61 11 50 10 59 70 73 41 55  3
 47 22 17]


In [9]:
# check we have the right indices
print( data['yr1_limi24.75']['dbname'].values[40],  given_list['name'][0] )

# check we have the right number of inds
print( len(ind), len(np.unique(ind)) )

agnddf_v1.4_10yrs agnddf_v1.4_10yrs.db
75 75


#### save the data in the order in which to save things

In [10]:
for yr_lim_tag in data:
    for colname in data[yr_lim_tag]:
        if colname != 'dbname':
             given_list[colname] = data[yr_lim_tag][colname].values[ind]

In [11]:
given_list

,name,Y1 effective survey area,median Y1 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area,Y3 effective survey area,median Y3 $i$-band coadded depth in effective survey area,std in Y3 $i$-band coadded depth in effective survey area,Y6 effective survey area,median Y6 $i$-band coadded depth in effective survey area,std in Y6 $i$-band coadded depth in effective survey area,Y10 effective survey area,median Y10 $i$-band coadded depth in effective survey area,std in Y10 $i$-band coadded depth in effective survey area
0,agnddf_v1.4_10yrs.db,14737.94,25.11,0.15,14942.84,25.71,0.13,14783.06,26.05,0.13,14666.66,26.32,0.12
1,alt_roll_mod2_dust_sdf_0.20_v1.4_10yrs.db,15353.15,25.02,0.14,14404.06,25.57,0.16,13888.84,25.95,0.13,14217.74,26.21,0.11
2,baseline_2snapsv1.4_10yrs.db,14255.87,25.05,0.14,14624.27,25.65,0.12,14302.25,26.00,0.11,14152.12,26.26,0.10
3,baseline_v1.4_10yrs.db,14677.83,25.09,0.15,14877.42,25.70,0.13,14565.63,26.03,0.12,14395.51,26.30,0.12
4,bulges_bs_v1.4_10yrs.db,15442.74,25.01,0.14,16086.69,25.63,0.13,15577.03,25.97,0.11,15280.39,26.24,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,wfd_depth_scale0.90_v1.4_10yrs.db,14665.87,25.09,0.14,14881.36,25.69,0.12,14656.74,26.03,0.12,14514.85,26.30,0.11
71,wfd_depth_scale0.95_noddf_v1.4_10yrs.db,14989.73,25.17,0.16,15140.70,25.78,0.14,15022.94,26.12,0.14,14975.04,26.39,0.13
72,wfd_depth_scale0.95_v1.4_10yrs.db,14709.88,25.10,0.14,14952.59,25.71,0.13,14773.56,26.05,0.12,14671.38,26.31,0.12
73,wfd_depth_scale0.99_noddf_v1.4_10yrs.db,15002.48,25.18,0.16,15115.15,25.79,0.14,15037.99,26.14,0.14,15011.92,26.40,0.13


In [12]:
# save the metrics
fname = 'static_metrics.csv'
given_list.to_csv('%s/%s' % (outdir, fname), index=False)